# Geohash + CID


In [ ]:
#!pip install pygeohash

In [1]:
import pygeohash as pgh
##load geojson
import geopandas as gpd
import pandas as pd
import os
import sys
sys.path.append("../")
from geohashtree.geohashtree import LiteTreeOffset, LiteTreeCID, FullTreeFile
from geohashtree.filesystem import ipfs_add_feature,ipfs_add_index_folder

Can only import/use native python functions.


In [2]:
asset = "dc_restaurants"
geojson_path = f"../assets/{asset}.geojson"
index_path = f"../data/geohash_offset_{asset}/index"

## Detached tree with offset
### Index Preparation
using one large file + lite index tree with offset

In [ ]:
offsettree = LiteTreeOffset()
offsettree.add_from_geojson(geojson_path)
offsettree.generate_tree_index(index_path)

### Onloading

In [ ]:
ipfs_add_index_folder(index_path)

In [5]:
index_cid = "bafybeib6ex2onm2wk5wxrdfe73l4m3la3c35nr5f4b4trbtidqaofduqiu"

In [6]:
ipfs_add_feature(geojson_path)

'bafybeia7jlkyzt22qgh66felldr322l5c7jdvm4xspr222f4usnd2ttbze'

In [ ]:
cid = "bafybeia7jlkyzt22qgh66felldr322l5c7jdvm4xspr222f4usnd2ttbze"

In [ ]:
##debugging
# import subprocess

# of,lg = (6266126, 4005)

# result = subprocess.run(['ipfs', 'cat', cid, "-o",str(of),"-l",str(lg)], stdout=subprocess.PIPE, check=True, text=True)

## Spatial query
Perform a spatial query and return a list of geohashes

In [3]:
geohashes = ['dqcjy',
 'dqcjz',
 'dqcjw',
 'dqcjv',
 'dqcjx',
 'dqcjq',
 'dqcjt',
 'dqcju',
 'dqcjr',
 'dqcjm',
 'dqcjs',
 'dqcjg',
 'dqcjk',
 'dqcnn',
 'dqcnq',
 'dqcnp',
 'dqcnj',
 'dqcnr',
 'dqcnm',
 'dqcnh',
 'dqcnk',
 'dqcmb',
 'dqcm8',
 'dqcm2',
 'dqcq0']

In [ ]:
geohashes = ['dqcnn', 'dqcjz', 'dqcjw', 'dqcjv', 'dqcnp', 'dqcjx', 'dqcjt', 'dqcnj']

## Feature Retrieval

In [ ]:
offsettree.query(geohashes,index_cid)

In [ ]:
offsettree.retrieve(geohashes,index_cid)

## Detached tree with CID

In [ ]:
#Entire workflow for LiteTreeCID

asset = "dc_restaurants"
geojson_path = f"../assets/{asset}.geojson"
index_path = f"../data/geohash_cid_{asset}/index"
pieces_path = f"../data/geohash_cid_{asset}"
cidtree = LiteTreeCID()
cidtree.add_from_geojson(geojson_path,pieces_path)

In [ ]:
cidtree.generate_tree_index(index_path)

In [ ]:
import glob
piece_files = glob.glob(pieces_path+"/*.geojson")

In [ ]:
ipfs_add_feature(piece_files)

In [ ]:
geojson_path

In [ ]:
geohashes = ['dqcnn', 'dqcjz', 'dqcjw', 'dqcjv', 'dqcnp', 'dqcjx', 'dqcjt', 'dqcnj']

In [ ]:
cids = cidtree.query(geohashes,index_path)

In [ ]:
len(cids)

In [ ]:
ret = cidtree.retrieve(geohashes,index_path)

In [ ]:
from geohashtree.geohashtree import FullTreeFile

## Attached tree with file

In [ ]:
#Entire workflow for FullTreeFile

asset = "dc_restaurants"
geojson_path = f"../assets/{asset}.geojson"
index_path = f"../data/geohash_full_{asset}/index"
fulltree = FullTreeFile()
fulltree.add_from_geojson(geojson_path)

In [ ]:
fulltree.features

In [ ]:
fulltree.generate_tree_index(index_path)

In [ ]:
ipfs_add_index_folder(index_path)

In [ ]:
geohashes = ['dqcnn', 'dqcjz', 'dqcjw', 'dqcjv', 'dqcnp', 'dqcjx', 'dqcjt', 'dqcnj']

In [ ]:
cid = "bafybeiez73pdnpevvoptzft54zmljj54di2pp44slk67wfeo7uz5hdbqoe/d/dq/dqc/dqcn/dqcnn"

In [ ]:
import os
os.path.join(cid,"dqcnn0.geojson")

In [ ]:
index_cid = "bafybeiez73pdnpevvoptzft54zmljj54di2pp44slk67wfeo7uz5hdbqoe"

In [ ]:
results = fulltree.query(geohashes,index_cid)

In [ ]:
len(results)

In [ ]:
results

In [ ]:
ret = cidtree.retrieve(geohashes,index_path)